# Example: Human segmentation with TransUnet and transfer learning from ImageNet-trained VGG16 model

In [1]:
import numpy as np
from glob import glob
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import load_model
from keras_unet_collection import models, utils
from sklearn.metrics import f1_score, jaccard_score

In [ ]:
# !pip install tensorflow-gpu==2.5.0

In [2]:
# !pip install keras-unet-collection

  Using cached keras_unet_collection-0.1.11-py3-none-any.whl (67 kB)


This example requires `keras-unet-collection`:
```
pip install keras-unet-collection
```

In [6]:
print(tf.__version__)

2.5.0


In [2]:
# the indicator of a fresh run
FIRST_TIME_RUNNING = False

# user-specified working directory
FILE_PATH = 'dataset/'
FILE_PATH_LABEL = 'dataset/'

## The Monuseg dataset

The dataset for this challenge was obtained by carefully annotating tissue images of several patients with tumors of different organs and who were diagnosed at multiple hospitals. This dataset was created by downloading H&E stained tissue images captured at 40x magnification from TCGA archive. 

H&E staining is a routine protocol to enhance the contrast of a tissue section and is commonly used for tumor assessment (grading, staging, etc.). 

Given the diversity of nuclei appearances across multiple organs and patients, and the richness of staining protocols adopted at multiple hospitals, the training datatset will enable the development of robust and generalizable nuclei segmentation techniques that will work right out of the box.

In [3]:
# train files path
path_train_img = FILE_PATH + 'train folder/img/'
path_val_mask = FILE_PATH_LABEL + 'train folder/mask/'

# validation file path
path_valid_img = FILE_PATH + 'validation folder/img/'
path_valid_mask = FILE_PATH_LABEL + 'validation folder/mask/'

# test files path
path_test_img = FILE_PATH + 'test folder/img/'
path_test_mask = FILE_PATH_LABEL + 'test folder/mask/'


In [4]:
train_input_names = np.array(sorted(glob(path_train_img+'*.png')))
train_label_names = np.array(sorted(glob(path_val_mask+'*.png')))
# flag_human = split_human_samples(train_label_names, human_id=0)
# train_input_names = train_input_names[flag_human]
# train_label_names = train_label_names[flag_human]

In [5]:
test_input_names = np.array(sorted(glob(path_test_img+'*.png')))
test_label_names = np.array(sorted(glob(path_test_mask+'*.png')))
# flag_human_test = split_human_samples(test_label_names, human_id=0)
# test_input_names = test_input_names[flag_human_test]
# test_label_names = test_label_names[flag_human_test]

In [6]:
valid_input_names = np.array(sorted(glob(path_valid_img+'*.png')))
valid_label_names = np.array(sorted(glob(path_valid_mask+'*.png')))

### Exploratory data analysis

In [7]:
def ax_decorate_box(ax):
    [j.set_linewidth(0) for j in ax.spines.values()]
    ax.tick_params(axis="both", which="both", bottom=False, top=False, \
               labelbottom=False, left=False, right=False, labelleft=False)
    return ax

In [8]:
I_MAX = 10   # explore 10 images
input_example = utils.image_to_array(train_input_names[:I_MAX], size=128, channel=3)
label_example = utils.image_to_array(train_label_names[:I_MAX], size=128, channel=1)

Text(0.5, 1.0, 'Segmentation mask')

In [9]:
i_example = 2

fig, AX = plt.subplots(1, 2, figsize=(7, 3))
plt.subplots_adjust(0, 0, 1, 1, hspace=0, wspace=0.1)

for ax in AX:
    ax = ax_decorate_box(ax)
AX[0].pcolormesh(np.mean(input_example[i_example, ...], axis=-1), cmap=plt.cm.gray)
AX[1].pcolormesh(label_example[i_example, ..., 0] > 0, cmap=plt.cm.gray)
AX[0].set_title("Original", fontsize=14)
AX[1].set_title("Segmentation mask", fontsize=14)

## Attention U-net with an ImageNet-trained backbone

Attention U-net is applied for this segmentation task. This architecture is modified from the conventionally used U-net by assigning attention gates on each upsampling level. 

Attention gates take upsampled (i.e., decoded) and downsampled (i.e., encoded) tensors as queries and keys, respectively. These queries and keys are mapped to intermediate channel sizes and fed into the additive attention learning. The resulting vector is rescaled by a sigmoid function and multiplied with the downsampled tensor (keys, but here treated as "values" of self-attention). The attention gate output replaces the downsampled tensor and is concatenated with the upsampled tensor.

Based on the amount and complexity of COCO samples, ImageNet-trained VGG16 is applied as an encoder backbone. This transfer learning strategy is expected to improve the segmentation performance based on two reasons: 

 * The ImageNet and COCO containts (somewhat) similar kinds of natural images with a high overlap of data distribution; 

 * The VGG16 architecture is a combination of same-padding convolution and max-pooling kernels, capable of extracting hierarchical features that can be processed by attention gates (ResNet backbone contains zero padding layers and is suboptimal in this case).

The code cell below configures the attention U-net with an ImageNet-trained VGG16 backbone. Hyper-parameters are explained through the Python helper function:

```python
from keras_unet_collection import models

help(models.att_unet_2d)
```

In [120]:

model = models.att_unet_2d((128, 128, 3), filter_num=[64, 128, 256, 512, 1024], n_labels=2,
                           stack_num_down=2, stack_num_up=2, activation='ReLU',
                           atten_activation='ReLU', attention='add', output_activation='Sigmoid',
                           batch_norm=True, pool=False, unpool=False,
                           backbone='VGG16', weights='imagenet',
                           freeze_backbone=True, freeze_batch_norm=True,
                           name='attunet')

The second layer of the configured model, i.e., right after an input layer, is expected to be the VGG16 backbone.

In [27]:
model.layers[1].name

'transunet_down0_0'

For simplicity, this segmentation model is trained with cross-entropy loss with SGD optimizer and a learning rate of 1E-2.

## Create Keras metric

In [21]:
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=1e-2), metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

## Training

The segmentation model is trained with 200 epoches with early stopping. Each epoch containts 100 batches and each batch contains 32 samples.

*The training process here is far from systematic, and is provided for illustration purposes only.*

In [22]:
def input_data_process(input_array):
    '''converting pixel vales to [0, 1]'''
    return input_array/255.

def target_data_process(target_array):
    '''Converting human, non-human pixels into two categories.'''
    target_array[target_array > 0] = 1  # grouping all other non-human categories 
    return keras.utils.to_categorical(target_array, num_classes=2)

In [23]:
L = len(train_input_names)
ind_all = utils.shuffle_ind(L)

L_train = int(0.9*L) 
L_valid = L - L_train
ind_train = ind_all[:L_train]; ind_valid = ind_all[L_train:]

In [24]:
valid_input = input_data_process(utils.image_to_array(valid_input_names, size=128, channel=3))
valid_label = target_data_process(utils.image_to_array(valid_label_names, size=128, channel=1))

In [25]:
N_EPOCH = 200  # number of epoches
N_BATCH = 100  # number of batches per epoch
N_SAMPLE = 32  # number of samples per batch

tol = 0  # current early stopping patience
max_tol = 2  # the max-allowed early stopping patience
min_del = 0  # the lowest acceptable loss value reduction 

# loop over epoches
for epoch in range(N_EPOCH):
    # initial loss record
    if epoch == 0:
        y_pred = model.predict([valid_input])
        record = np.mean(keras.losses.binary_crossentropy(valid_label, y_pred))
        print('\tInitial loss = {}'.format(record))
    # loop over batches
    for step in range(N_BATCH):
        # selecting smaples for the current batch
        ind_train_shuffle = utils.shuffle_ind(L_train)[:N_SAMPLE]
        # batch data formation
        # augmentation is not applied
        train_input = input_data_process(utils.image_to_array(train_input_names[ind_train_shuffle], size=128, channel=3))
        train_label = target_data_process(utils.image_to_array(train_label_names[ind_train_shuffle], size=128, channel=1))
        # train on batch
        loss_ = model.train_on_batch([train_input,], [train_label,])
        # ** training loss is not stored ** #
    # epoch-end validation
    y_pred = model.predict([valid_input])
    record_temp = np.mean(keras.losses.binary_crossentropy(valid_label, y_pred))
    # ** validation loss is not stored ** #
    # if loss is reduced
    if record - record_temp > min_del:
        print('Validation performance is improved from {} to {}'.format(record, record_temp))
        record = record_temp  # update the loss record
        tol = 0  # refresh early stopping patience
        # ** model checkpoint is not stored ** #
    # if loss not reduced
    else:
        print('Validation performance {} is NOT improved'.format(record_temp))
        tol += 1
        if tol >= max_tol:
            print('Early stopping')
            break
        else:
            # Pass to the next epoch
            continue

	Initial loss = 0.6878138184547424


In [34]:
model.save('attention-unet-monuseg.h5')

## Evaluation

The testing set performance is evaluated with cross-entropy and example outputs.

In [77]:
model = load_model('attention-unet-monuseg.h5', custom_objects={'mean_iou': mean_iou})

In [78]:
test_input = input_data_process(utils.image_to_array(test_input_names, size=128, channel=3))
test_label = target_data_process(utils.image_to_array(test_label_names, size=128, channel=1))

In [79]:
# prediction on test_input without threshold
# y_pred = model.predict([test_input], verbose=1)
y_pred = model.predict(test_input)

# prediction on test_input with threshold
y_pred_t = (y_pred > 0.5).astype(np.uint8)

In [80]:
print(y_pred[0].shape)

(128, 128, 2)


In [81]:
print(test_input[0].shape)

(128, 128, 3)


In [82]:
print('Testing set cross-entropy = {}'.format(np.mean(keras.losses.binary_crossentropy(test_label, y_pred))))

Testing set cross-entropy = 0.4505641758441925


**Example of outputs**

As a common practice in computer vision projects, only nice looking samples are plotted : |

Text(0.5, 1.0, 'Labeled truth')

In [83]:
i_sample = 11

fig, AX = plt.subplots(1, 4, figsize=(13, (13-0.2)/4))
plt.subplots_adjust(0, 0, 1, 1, hspace=0, wspace=0.1)
for ax in AX:
    ax = ax_decorate_box(ax)
AX[0].pcolormesh(np.mean(test_input[i_sample, ...], axis=-1), cmap=plt.cm.gray)
# AX[1].pcolormesh(y_pred[i_sample, ..., 0], cmap=plt.cm.jet)
# AX[2].pcolormesh(test_label[i_sample, ..., 0], cmap=plt.cm.jet)


AX[1].pcolormesh(y_pred[i_sample, ..., 0], cmap=plt.cm.gray)
AX[2].pcolormesh(test_label[i_sample, ..., 0], cmap=plt.cm.gray)
AX[3].pcolormesh(y_pred_t[i_sample, ..., 0], cmap=plt.cm.gray)


AX[0].set_title("Original", fontsize=14)
AX[1].set_title("Pixels belong to human (red for high probabilities)", fontsize=14)
AX[2].set_title("Labeled truth", fontsize=14)


In [85]:
print(y_pred_t[0].shape)

(128, 128, 2)


In [86]:
Image.fromarray(255*np.array(y_pred_t[0])).resize((128, 128)).save('mask.png')

In [88]:
print(test_input_names)

['dataset/test folder/img\\0001.png' 'dataset/test folder/img\\0002.png'
 'dataset/test folder/img\\0003.png' 'dataset/test folder/img\\0004.png'
 'dataset/test folder/img\\0005.png' 'dataset/test folder/img\\0006.png'
 'dataset/test folder/img\\0007.png' 'dataset/test folder/img\\0008.png'
 'dataset/test folder/img\\0009.png' 'dataset/test folder/img\\0010.png'
 'dataset/test folder/img\\0011.png' 'dataset/test folder/img\\0012.png'
 'dataset/test folder/img\\0013.png' 'dataset/test folder/img\\0014.png']


In [50]:
!pip install scikit-learn

In [99]:
f1, iou = [], []
for i in range(len(test_input_names)):
    INPUT = np.array(Image.open(test_label_names[i]))
    INPUT = np.where(INPUT >= 127, 1, 0)
    LABEL = np.array(Image.open(test_label_names[i]))
    LABEL = np.where(LABEL >= 127, 1, 0)
    y_true, y_pred = INPUT.flatten(), LABEL.flatten()
    f1.append(f1_score(y_true, y_pred))
    iou.append(jaccard_score(y_true, y_pred))

In [100]:
np.mean(f1), np.mean(iou)

(1.0, 1.0)